In [1]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string

In [22]:
df = pd.read_csv(r"C:\Users\saich\Downloads\English-Telugu (2).csv")
df.shape

(1105, 2)

In [23]:
# English --> Encoder ---> longtermsequences, shortterm sequences --> concat 
# concat --> Decoder ---> target column

In [24]:
df["Telugu"] = "start" + " " + df["Telugu"] + " " + "end"

In [25]:
# convert the text to words
# fit the tokenizer on words


df["English" ] = df["English"].apply(lambda iterable: word_tokenize(iterable))
df["Telugu" ] = df["Telugu"].apply(lambda iterable: word_tokenize(iterable))

In [26]:
english_tokenizer = Tokenizer()
telugu_tokenizer = Tokenizer()

0
0


In [28]:
english_tokenizer.fit_on_texts(df["English"])
telugu_tokenizer.fit_on_texts(df["Telugu"])

In [29]:
print(len(english_tokenizer.word_index))
print(len(telugu_tokenizer.word_index))

1513
2285


In [30]:
english_sequences = english_tokenizer.texts_to_sequences(df["English"])
telugu_sequences = telugu_tokenizer.texts_to_sequences(df["Telugu"])

In [18]:
# # padding based on maximum lenght of two sequences

# english_sequences = pad_sequences(english_sequences,padding="post")

In [19]:
# print(english_sequences.shape)
# print(telugu_sequences.shape)

(1105, 31)
(1105, 29)


In [34]:
print(telugu_sequences[0])
print(telugu_sequences[0][:-1]) # decoder input
print(telugu_sequences[0][1:]) # decoder output
# 1 represents start
# 2 reprensents end

[1, 37, 574, 575, 51, 3, 2]
[1, 37, 574, 575, 51, 3]
[37, 574, 575, 51, 3, 2]


In [40]:
decoder_inputs = [seq for seq in telugu_sequences[:-1]]
decoder_inputs = pad_sequences(decoder_inputs,padding="post")
decoder_outputs = [seq for seq in telugu_sequences[1:]]
decoder_outputs = pad_sequences(decoder_outputs,padding="post")

In [44]:
decoder_outputs = keras.utils.to_categorical(decoder_outputs,num_classes=2286)

print(decoder_inputs.shape)
print(decoder_outputs.shape)

(1104, 29)
(1104, 29, 2286)


In [ ]:
# we will have to create input for decoder and output for decoder

# input_decoder --> remove end from the sequences
# output_decoder --> remove start from the sequences

In [46]:
# Encoder Part

encoder_inputs = keras.Input(shape=(None, 1514))
encoder = keras.layers.LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder Part
decoder_inputs = keras.Input(shape=(None, 2286))
decoder_lstm = keras.layers.LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(2286, activation="softmax")
decoder_outputs = decoder_dense(2286)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()

ValueError: Cannot convert '1514' to a shape.